In [1]:
import json
import requests
import pandas as pd
import io

# Step 1 Read Meta Data  & End Points 

In [2]:
def get_ids(start, end):
    years = [str(x) for x in range(start, end+1)]
    ids = {"General": [], "Research": [], "Ownership": []}
    url = 'https://openpaymentsdata.cms.gov/api/1/metastore/schemas/dataset/items?show-reference-ids'
    data = requests.get(url).json()
    for d in data:
        split_title = d['title'].split(" ")
        identifier = d['identifier']
        if split_title[0] in years:
            if split_title[1] in ids:
                ids[split_title[1]].append(identifier)
    return ids

In [3]:
ids = get_ids(2020, 2021)

In [4]:
usecols=["Date_of_Payment", "Product_Category_or_Therapeutic_Area_1", "Recipient_State", "Recipient_City", "Covered_Recipient_Type", "Total_Amount_of_Payment_USDollars"]

In [5]:
def get_datasets(id_list):
    for category, category_ids in id_list.items():
        for category_id in category_ids:
            url = 'https://openpaymentsdata.cms.gov/api/1/metastore/schemas/dataset/items/{}'.format(category_id)
            data = requests.get(url).json()
            download = data["distribution"][0]["downloadURL"]
            try:
                response = requests.get(download)
                df = pd.read_csv(io.BytesIO(response.content), nrows=10)
                print(category, category_id)
                print(len(df))
            except Exception as e:
                print("Failed to download:", download, "Error:", e)

In [6]:
df = get_datasets(ids)

General a08c4b30-5cf3-4948-ad40-36f404619019
10
General 0380bbeb-aea1-58b6-b708-829f92a48202
10
Research ce1d28dd-0094-5060-a036-580329439600
10
Research 9c248e7e-7c7f-478b-ab84-ce0919d72c1c
10
Ownership b0c03b8d-06df-58f2-8ce2-4daeffee147e
10
Ownership a9a0bf48-6b96-4589-b4c2-3c5dcfbeaca2
10
